### Attempt I

In [10]:
module facility_location

using JuMP
using Cbc


function solve_facility_location()

    n = 5 # number of locations
    m = 6 # number of customers
    # i: customers, j : locations, h: demand, f = fixed component of warehouse cost based on size, v= capacity based on warehouse size  

    h = [320000 200000 160000 220000 350000 175000];

    f = [
        300000 250000 220000 220000 240000 
        500000 420000 375000 375000 400000
        ];
#     v = [
#         200000 200000 200000 200000 200000
#         400000 400000 400000 400000 400000
#         ];

# v  = [200000 400000]

    c= [
        2.0 2.5 3.5 4.0 5.0 5.5
        2.5 2.5 2.5 3.0 4.0 4.5
        3.5 3.5 2.5 2.5 3.0 3.5
        4.0 4.0 3.0 2.5 3.0 2.5
        4.5 5.0 3.0 3.5 2.5 4.0
        ];

    model = Model(Cbc.Optimizer);

    @variable(model, x[1:n] >= 0, Bin);     # binary: 1 for yes to location, 0 for no to location
    @variable(model, s[1:n] >= 0, Bin);      # binary: 1 for yes to large DC, 0 to small DC,valid only when x = 1
    @variable(model, y[1:m,1:n] >= 0);

    @objective(model, Min, sum( f[s[j]+1,j]*x[j] for j in 1:n) + sum( sum( h[i]*c[j,i]*y[i,j] for j in 1:n) for i = 1:m));

    @constraint(model,[i = 1:m], sum(y[i,j] for j in 1:n) == 1);
    @constraint(model,[i = 1:m, j = 1:n], y[i,j] <= x[j]);
    @constraint(model,[j = 1:n], sum(h[i]*y[i,j] for i in 1:m)<= 2000000*(x[j]*(s[j]+1)))
    @constraint(model,[j = 1:n], s[j] <= x[j])

    #-------
    # SOLVE
    #-------

    optimize!(model)

    println();
    for j = 1:n
        if (value(x[j]) == 1)
            println("facility ",j,"size",v[i,j]);
        end
    end

end

solve_facility_location();

end


LoadError: ArgumentError: invalid index: s[1] + 1 of type JuMP.AffExpr

### Attempt II

In [16]:
using JuMP
# using Cbc
using Gurobi

Demand = ["Northwest", "Southwest", "Upper Midwest", "Lower Midwest", "Northeast", "Southeast"]
I = length(Demand) # number of customers : 6
    
Facilities = ["Seattle", "Denver", "St. Louis", "Atlanta", "Philadelphia"]
J = length(Facilities) # number of locations : 5
    
# i: customers, j : locations, h: demand, f = fixed component , v= capacity based on warehouse size
Years = ["2007", "2008", "2009", "2010", "2011"]
P = length(Years) #number of years
    
Size = ["Small", "Large"]
S = length(Size) # number of warehouse sizes
    
    # new demand after 80% growth

"Demand_old= [
        320000 200000 160000 220000 350000 175000
        576000 360000 288000 396000 630000 315000
        1036800 648000 518400 712800 1134000 567000
        1866240 1166400 933120 1283040 2041200 1020600
        1866240 1166400 933120 1283040 2041200 1020600];" #calculated based on 80% growth for the next three years

Demand= [
        320000 576000 1036800 1866240 1866240 
        200000 360000 648000 1166400 1166400
        160000 288000 518400 933120 933120
        220000 396000 712800 1283040 1283040
        350000 630000 1134000 2041200 2041200
        175000 315000 567000 1020600 1020600
];

growth = 1.8

    # Fixed cost based on size of warehouse 
"Fixed_old = [
        300000 250000 220000 220000 240000 
        500000 420000 375000 375000 400000 ];"

Fixed = [ 
    300000 500000
    250000 420000
    220000 375000
    220000 375000
    240000 400000    
];

# Variable cost
"Var = [
        0.20 0.20 0.20 0.20 0.20
        0.20 0.20 0.20 0.20 0.20  ];"
    
Capacity = [2000000 4000000] # s as looping variable with value from 1:S
# S = length(Capacity) = 2

InvFixed = 475000

InvVar = 0.165

"ShippingCost_Old = [
        2.0 2.5 3.5 4.0 5.0 5.5
        2.5 2.5 2.5 3.0 4.0 4.5
        3.5 3.5 2.5 2.5 3.0 3.5
        4.0 4.0 3.0 2.5 3.0 2.5
        4.5 5.0 3.0 3.5 2.5 4.0];"

ShippingCost = [
    2.0 2.5 3.5 4.0 4.5
    2.5 2.5 3.5 4.0 5.0
    3.5 2.5 2.5 3.0 3.0
    4.0 3.0 2.5 2.5 3.5
    5.0 4.0 3.0 3.0 2.5
    5.5 4.5 3.5 2.5 4.0
];
    
model = Model(Gurobi.Optimizer);

    # binary: decision to open a facility of size s in location j in year p
@variable(model, x[j = 1:J, s = 1:S, p = 1:P] >= 0, Bin); 
    # x[j,s,p]

    # Fraction of demand from zone i to allocate to facility in location j in year p
@variable(model,y[i = 1:I, j = 1:J, p = 1:P] >=0); 
    # y[i,j,p]

    # Tracking the year when the facility was first opened
@variable(model, z[j = 1:J, p = 1:P] >=0, Bin); 
    # z[j,p]

    #-------

    #
@objective(model, Min, 
sum(x[j,s,p] * (Fixed[j,s] + InvFixed) for j in 1:J, s in 1:S, p in 1:P) + 
sum(y[i,j,p,s] * Demand[i,p] * (0.2 + InvVar + ShippingCost[i,j]) for s in 1:S, i in 1:I, j in 1:J, p in 1:P));
    
    #-------

    # edited
    # equation (2)
    # Cannot open a small and large facility in the same location    
@constraint(model, [j = 1:J, p = 1:P], sum(x[j,s,p] for s in 1:S) <= 1); 

    # edited
    # equation (3)
    # Must update the tracking variable z if a facility f was not open in the previous year
@constraint(model, [j = 1:J, p = 2:P], z[j,p] <= sum(x[j,s,p] for s in 1:S) - sum(x[j,s,p-1] for s in 1:S));

    # new
    # equation (4)
    # If a facility is leased, the lease must be of minimum 3 years length
@constraint(model, [j = 1:J], sum(z[j,p] for p in 1:P) <= sum(x[j,s,p] for s in 1:S, p in 1:P));

    # edited
    # equation (5) - need to correct constrint equation in Overleaf
    # can only assign demand to open facilities
@constraint(model, [i = 1:I, j = 1:J, p = 1:P], y[i,j,p] <= sum(x[j,s,p] for s in 1:S));  

    # edited 
    # equation (6) - need to correct constrint equation in Overleaf
    # no unassigned demand for any year
@constraint(model, [p = 1:P], sum(sum(y[i,j,p] for j in 1:J) for i in 1:I) == 1 );

    # edited
    # equation (7)
    # Capacity constraint (Cannot allocate more demand to a facility j than there is capacity for)
@constraint(model, [j = 1:J, p = 1:P], sum(y[i,j,p]*Demand[i,p] for i in 1:I) <= sum(x[j,s,p]*Capacity[s] for s in 1:S));

    # edited
    # equation (8)
    # non-negativity constraint, cannot allocate negative demand
@constraint(model, [i = 1:I, j = 1:J, p = 1:P], y[i,j,p] >= 0);

    # @constraint(model,yr > 1 ? [j=1:m,yr = 1:P],  (z[j,yr] <= sum(x[j,c,yr] for c in 1:s)-sum(x[j,c,yr-1] for c in 1:s)) : );
    #  @constraint(model, [i=1:m,j=1:n,yr=1:P], yr <= 4 ? sum(x[j,c,yrj+1] = sum x[j] for j in 3:n | 0)
    #  @constraint(model,[j=1:n,z=1:3],(sum(x[j,c,yr] for yr in z:z+2)== 0 || sum(x[j,c,yr] for yr in z:z+2)== 3))
    #  @constraint(model,[j=1:n,c=1:s,z=1:3],(sum(x[j,c,yr] for yr in z:z+2)== 3))

    #-------
    # SOLVE
    #-------

optimize!(model)

println();

for j = 1:J
    for p = 1:P
        for s = 1:S
            if (value(x[j,s,p]) == 1)
                if (value(z[j,p]) == 1)
                    println("facility: ", Facilities[j]," size: ", Size[s], " year: ", Years[p]);
                end
            end
        end
    end
    
end


Academic license - for non-commercial use only - expires 2022-10-21
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 380 rows, 225 columns and 1175 nonzeros
Model fingerprint: 0xf9e35a4b
Variable types: 150 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [7e+05, 2e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 195 rows and 40 columns
Presolve time: 0.00s
Presolved: 185 rows, 185 columns, 710 nonzeros
Variable types: 150 continuous, 35 integer (35 binary)

Root relaxation: objective 1.941842e+07, 40 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.9418e+07    0    8          - 1.9418e+07      -     -    0s
H    0     0                    1

LoadError: syntax: unexpected "end"